<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/main/Mixed_Integer_Problem_for_Draft_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Initialisierung und Konfiguration

## 1.1 Bibliotheken installieren und laden


In [ ]:
import pandas as pd
import numpy as np
import re
import random
!pip install mip
from mip import Model, BINARY, CONTINUOUS, xsum, maximize
!pip install rapidfuzz
from rapidfuzz import process

## 1.2 Liga-Parameter und Rahmenbedingungen definieren

In [ ]:
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# 2 Datenbeschaffung und -vorverarbeitung


## 2.1 ADP, Bye Weeks, Projections und reale Daten laden

In [ ]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)


bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)


season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
      .drop_duplicates(subset='Player', keep='first')
)
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)
week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,
        season_projections['avg_proj']
    )
season_projections = season_projections.drop(columns='avg_proj')

real_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/real_projections/real_projections_{year}.csv'
real_projections = (
    pd.read_csv(real_projections_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'points': 'Projection', 'week': 'Week'})
      .loc[:, ['Player', 'POS', 'Projection', 'Week']]
      .pivot_table(
         index=['Player', 'POS'],
         columns='Week',
         values='Projection',
         aggfunc='first'
      )
      .rename_axis(None, axis=1)
      .add_prefix('Week_')
      .reset_index()
      .fillna(0)
)

real_data_RZ_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/weekly_data_{year}.csv'
real_data_RZ = (
    pd.read_csv(real_data_RZ_url)
    .rename(columns={'position': 'POS', 'player_display_name': 'Player'})
    .replace({'FB': 'RB'})
    .drop(columns = ['season'], errors='ignore')
    .loc[lambda real_data_RZ: real_data_RZ['POS'].isin(allowed_positions)]
)


real_data_FP_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/Weekly_Data/FantasyPros_Fantasy_Football_Points_PPR_{year}.csv'
real_data_FP = (
    pd.read_csv(real_data_FP_url)
      .rename(columns={'Pos': 'POS', 'player_display_name': 'Player'})
      .replace({'-': 0, 'BYE': 0})
      .drop(columns = ['#', 'Team'], errors='ignore')
      .query("POS in ['K', 'DST']")
)

real_data_FP = real_data_FP.melt(
    id_vars=['Player', 'POS'],
    value_vars=[str(w) for w in range(1, 19)],
    var_name='week',
    value_name='fantasy_points_ppr'
)
real_data_FP = real_data_FP[~real_data_FP['fantasy_points_ppr'].isin(['BYE', '-', None])].copy()
real_data_FP['fantasy_points_ppr'] = pd.to_numeric(real_data_FP['fantasy_points_ppr'])

real_data = pd.concat([real_data_FP, real_data_RZ], ignore_index=True)
real_data['week'] = real_data['week'].astype(int)

print("Positions in Fantasy Pros Data:")
print(real_data_FP['POS'].value_counts())
print("\nPositions in Row Zero Data:")
print(real_data_RZ['POS'].value_counts())
print("\nPositions in Concatonated Data")
print(real_data['POS'].value_counts())

## 2.2 Spieler Namen verschiedener Quellen angleichen

In [ ]:
adp_names = player_adp['Player'].unique().tolist()
adp_pos_dict = dict(zip(player_adp['Player'], player_adp['POS']))

def match_name_with_pos(name, pos, reference_list, reference_pos_dict, cutoff=85.5):
    filtered_candidates = [p for p in reference_list if reference_pos_dict.get(p) == pos]
    result = process.extractOne(name, filtered_candidates, score_cutoff=cutoff)
    return result[0] if result else None

real_data['Player'] = real_data.apply(
    lambda row: match_name_with_pos(row['Player'], row['POS'], adp_names, adp_pos_dict),
    axis=1
)
real_data = real_data.drop_duplicates(subset=['Player', 'POS', 'week'], keep='first')

real_data = real_data[real_data['Player'].notnull()].copy()

## 2.3 Berechnung von TTL, Dropoff und VOR

In [ ]:
dropoff_w = {'QB':1.0,'RB':1.0,'WR':1.0,'TE':0.9,'K':0.4,'DST':0.3}
vor_w     = {'QB':0.8,'RB':1.0,'WR':1.0,'TE':0.8,'K':0.25,'DST':0.25}

rep_ttl = (
    season_projections
      .groupby('POS')['TTL']
      .apply(lambda s: s.nlargest(lineup_req[s.name]*num_teams)
                   .iloc[-1]
             if len(s) >= lineup_req[s.name]*num_teams else 0)
)

season_projections = (
    season_projections
      .sort_values(['POS','TTL'], ascending=[True,False])
      .assign(
          dropoff=lambda df: (
              df.groupby('POS')['TTL']
                .diff(-1)
                .fillna(0)
                .mul(df['POS'].map(dropoff_w))
          ),
          VOR=lambda df: (
              (df['TTL'] - df['POS'].map(rep_ttl))
                .clip(lower=0)
                .mul(df['POS'].map(vor_w))
          )
      )
      .fillna({'dropoff': 0, 'VOR': 0, 'ADP': 999})
      .sort_values(['ADP', 'TTL'], ascending=[True, False])
      .reset_index(drop=True)
)


# 3 Modellparameter, Variablenaufbau und Optimierungsproblem

In [ ]:
# 1 Initalisierung
players            = (season_projections[['Player', 'POS', 'ADP', 'TTL', 'dropoff', 'VOR']]
                      .copy()
                      .sort_values(['ADP', 'TTL'], ascending=[True, False])
                      .reset_index(drop=True))
players['Rank']    = players.index + 1

players_list       = players['Player'].dropna().copy().tolist()
positions          = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible      = {'RB', 'WR', 'TE'}
weeks              = list(range(1, 18))

pos                = dict(zip(season_projections['Player'], season_projections['POS']))
lineup_req         = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
max_req            = {'QB': 2, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
week_cols          = [col for col in season_projections.columns if col.startswith('Week_')]

season_projections_dict = {
	row['Player']: {
		**{int(week.replace('Week_', '')): row[week] for week in week_cols},
		'dropoff': row['dropoff'],
		'VOR':     row['VOR']
	}
	for _, row in season_projections.iterrows()
}

beta               = {t: 120.0 for t in weeks}
alpha              = 0.3
lambda_0           = 1
lambda_1           = 1
lambda_2           = 2
lambda_3           = 5
lambda_4           = 0.25
df_sorted          = players.sort_values('Rank').reset_index(drop=True)
topk_pct           = 0.0025

# 2 Festlegen der Draft Reihenfolge

teams              = [f'Team {i+1}' for i in range(num_teams)]
DM_team            = random.choice(teams)
draft_order        = []
for rnd in range(num_rounds):
	order = teams if rnd % 2 == 0 else teams[::-1]
	draft_order += order

# 3 Heuristische Gegner Draft Funktion

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
	remaining_players = sorted(available, key=lambda p: Rk[p])
	topk              = max(1, int(len(remaining_players) * topk_pct))

	deficits = {
		j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
		for j in lineup_req if j != 'FLEX'
	}
	needed = [j for j, d in deficits.items() if d > 0]

	if needed:
		candidates = [p for p in remaining_players if pos[p] in needed]
		pool       = candidates[:topk] if len(candidates) >= topk else candidates
		if pool:
			return random.choice(pool)

	return random.choice(remaining_players[:topk])

# 4 Draft Initialisierung

rosters             = {tm: [] for tm in teams}
available           = set(players_list)
draft_log           = []


# 5 Draft Loop mit MIP

for pick_idx, team in enumerate(draft_order, start=1):

    for p in available:
        ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
        if len(ranks) != 1:
            print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    rank_dict = dict(zip(df_sorted['Player'], df_sorted['Rank']))
    remaining_players = sorted(available, key=lambda p: rank_dict.get(p, float('inf')))
    Rk = {p: i + 1 for i, p in enumerate(remaining_players)}
    player_vars       = set(remaining_players) | set(rosters[team])
    picks_remaining   = num_rounds - len(rosters[team])

    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in remaining_players}
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        m.objective = (
            lambda_0    * xsum(season_projections_dict[i][t] * x[i, t]        for i in remaining_players for t in weeks)
            + lambda_1  * xsum(z[t]                                           for t in weeks[:15])
            + lambda_2  * xsum(z[t]                                           for t in weeks[15:])
            + lambda_3  * xsum(season_projections_dict[i]['dropoff'] * y[i]   for i in remaining_players)
            - lambda_4  * xsum(season_projections_dict[i]['VOR'] * y[i]       for i in remaining_players)
        )

        m += xsum(y[i] for i in remaining_players) == picks_remaining

        for pos_name, req in lineup_req.items():
            if pos_name != 'FLEX':
                already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
                need = max(0, req - already_satisfied)
                m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        for t in weeks:
            for j in positions - {'FLEX'}:
                m += xsum(y[i] for i in remaining_players if pos[i] == j) <= max_req[j]
            for i in remaining_players:
                m += (
                    (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
                    (x[i, t] if pos[i] in flex_eligible else 0)
                    <= 1
                )
        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]
        for t in weeks:
            m += z[t] <= xsum(season_projections_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        n_k = pick_idx
        for future_pick in range(pick_idx + 1, pick_idx + picks_remaining * len(teams), len(teams)):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        m.optimize()
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(season_projections_dict[i][t] for t in weeks)
                vor = season_projections_dict[i]['VOR']
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    else:
        current_round   = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster    = rosters[team],
            available = available,
            Rk        = Rk,
            lineup_req= lineup_req,
            topk_pct  = dynamic_topk_pct
        )

    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick':    pick_idx,
        'Team':    team,
        'Player':  pick,
        'Round':   (pick_idx - 1) // len(teams) + 1,
        'POS':     pos[pick]
    })

# 6 Ergebnisse zusammenführen

df_draft = pd.DataFrame(draft_log)
print(m.status)
print(m)


# 4 Ergebnisaufbereitung: Draft-Log exportieren und zusammenführen

In [ ]:
# Gesamtprojektion pro Team
team_ttl_proj = (
    df_draft
    .merge(season_projections[['Player','TTL']], on='Player', how='left')
    .assign(TTL=lambda df: df['TTL'].fillna(0))
    .groupby('Team')['TTL']
    .sum()
    .reset_index(name='TTL_proj')
)
print(team_ttl_proj)

# Positionen zählen
position_counts = df_draft.pivot_table(
    index='Team', columns='POS', aggfunc='size', fill_value=0
)
print(position_counts)

# Draft-Infos pro Team
result_dfs = {
    team: (
        df_draft[df_draft['Team']==team]
        .sort_values('Pick')
        .assign(Pick_Info=lambda df:
            'Round '+df['Round'].astype(str)+' Pick '+df['Pick'].astype(str))
        [['Player','Pick_Info','POS']]
    )
    for team in df_draft['Team'].unique()
}

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()


# 6 Evaluation

## 6.1 Reale projections laden

In [ ]:
roster_projections = (
    df_draft
      .merge(
         real_projections.drop(columns=['POS']),
         on='Player',
         how='left'
      )
      .sort_values(['Team','Pick'])
      .sort_values(by=['Team', 'Pick'])
      .reset_index(drop=True)
)

## 6.2 Liga Schedule aufbauen

In [ ]:
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)

## 6.3 Line-Up anhand greedy Ansatz aufstellen

In [ ]:
# Roster limits
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

# Für jede Woche und jedes Team die beste Aufstellung bestimmen
def get_best_lineup(team, week, roster_projections, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team]
    lineup = []
    used_players = set()

    for pos, limit in lineup_req.items():
        if pos != 'FLEX':
            candidates = team_roster[team_roster['POS'] == pos]
        else:
            candidates = team_roster[
                (team_roster['POS'].isin(['RB', 'WR', 'TE'])) &
                (~team_roster['Player'].isin(used_players))
            ]

        starters = candidates.sort_values(week_col, ascending=False).head(limit)
        lineup.append(starters)
        used_players.update(starters['Player'])

    return pd.concat(lineup)


# Punkte aus real_data holen
def get_actual_points(lineup, week, real_data):
    merged = lineup.merge(
        real_data[real_data['week'] == week],
        left_on='Player', right_on='Player', how='left'
    )
    # Fülle fehlende Werte (z.B. bei Bye Weeks) mit 0
    merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
    return merged['fantasy_points_ppr'].sum()

# Für alle Wochen und alle Matchups durchlaufen
results = []
for week_idx, matchups in enumerate(reg_schedule, 1):
    for team1, team2 in matchups:
        lineup1 = get_best_lineup(team1, week_idx, roster_projections, lineup_req)
        lineup2 = get_best_lineup(team2, week_idx, roster_projections, lineup_req)
        points1 = get_actual_points(lineup1, week_idx, real_data)
        points2 = get_actual_points(lineup2, week_idx, real_data)
        winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
        loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
        results.append({
            'Woche': week_idx,
            'Team 1': team1,
            'Team 2': team2,
            'Punkte Team 1': points1,
            'Punkte Team 2': points2,
            'Sieger': winner,
            'Verlierer': loser
        })
df_results = pd.DataFrame(results)

## 6.4 Analyse des Line-Up

In [ ]:
def get_lineup_and_bench(team, week, roster_projections, real_data, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team].copy()

    lineup = get_best_lineup(team, week, roster_projections, lineup_req)

    lineup_players = set(lineup['Player'])
    bench = team_roster[~team_roster['Player'].isin(lineup_players)].copy()

    week_actuals = real_data[real_data['week'] == week][['Player', 'fantasy_points_ppr']]
    week_actuals = week_actuals.rename(columns={'fantasy_points_ppr': 'Actual_Points'})

    def process_df(df):
        df['Projected_Points'] = df[week_col]
        df.drop(columns=[col for col in df.columns if col.startswith("Week_")], inplace=True)
        df = df.merge(week_actuals, on='Player', how='left')
        df['Actual_Points'] = df['Actual_Points'].fillna(0)
        return df

    lineup = process_df(lineup)
    bench = process_df(bench)

    return lineup, bench

lineup, bench = get_lineup_and_bench(DM_team, 1, roster_projections, real_data, lineup_req)

pd.set_option('display.max_rows', 30)

print("=== Lineup (Week 1) ===")
display(lineup[['Player', 'POS', 'Projected_Points', 'Actual_Points']])

print("=== Bench (Week 1) ===")
display(bench[['Player', 'POS', 'Projected_Points', 'Actual_Points']])


## 6.5 Auspielen der Liga inklusive Playoffs

In [ ]:
# Deine vorhandene Liste mit Teams
teams = [f'Team {i+1}' for i in range(num_teams)]

# Sieger und Verlierer zählen (Unentschieden ausschließen)
wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
losses = df_results[df_results['Verlierer'] != 'Unentschieden']['Verlierer'].value_counts()

# Draws zählen: alle Teams, die in einem Unentschieden beteiligt waren
draws = (
    df_results[df_results['Sieger'] == 'Unentschieden'][['Team 1', 'Team 2']]
    .stack()
    .value_counts()
)

points_for = pd.concat([
    df_results[['Team 1', 'Punkte Team 1']].rename(columns={'Team 1': 'Team', 'Punkte Team 1': 'Points'}),
    df_results[['Team 2', 'Punkte Team 2']].rename(columns={'Team 2': 'Team', 'Punkte Team 2': 'Points'})
])
points_for = points_for.groupby('Team')['Points'].sum()

# Punkte, die jedes Team kassiert hat ("Points Against")
points_against = pd.concat([
    df_results[['Team 1', 'Punkte Team 2']].rename(columns={'Team 1': 'Team', 'Punkte Team 2': 'Points'}),
    df_results[['Team 2', 'Punkte Team 1']].rename(columns={'Team 2': 'Team', 'Punkte Team 1': 'Points'})
])
points_against = points_against.groupby('Team')['Points'].sum()

# Zusammenführen in ein DataFrame
record = pd.DataFrame({'Team': teams})
record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
record['Losses'] = record['Team'].map(losses).fillna(0).astype(int)
record['Draws'] = record['Team'].map(draws).fillna(0).astype(int)
record['Points For'] = record['Team'].map(points_for).fillna(0)
record['Points Against'] = record['Team'].map(points_against).fillna(0)
record = record.sort_values(by=['Wins', 'Draws', 'Points For'], ascending=[False, False, False]).reset_index(drop=True)

In [ ]:
# Setze die Playoff-Wochen
playoff_weeks = [15, 16, 17]

# Teams nach Rang sortieren (wie zuvor)
ranked_teams = record['Team'].tolist()

# Woche 15: Seed 3 vs 6, Seed 4 vs 5
week_15_matchups = [
    (ranked_teams[2], ranked_teams[5]),  # Match 1
    (ranked_teams[3], ranked_teams[4])   # Match 2
]

# Ergebnisse Woche 15
week15_results = []
for team1, team2 in week_15_matchups:
    lineup1 = get_best_lineup(team1, 15, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 15, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 15, real_data)
    points2 = get_actual_points(lineup2, 15, real_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week15_results.append({
        'Woche': 15,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_15 = [r['Sieger'] for r in week15_results]

# Woche 16 Matchups:
# Match 3: Winner Match 1 vs Seed 2
# Match 4: Winner Match 2 vs Seed 1
week_16_matchups = [
    (winners_15[0], ranked_teams[1]),  # gegen Seed 2
    (winners_15[1], ranked_teams[0])   # gegen Seed 1
]

week16_results = []
for team1, team2 in week_16_matchups:
    lineup1 = get_best_lineup(team1, 16, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 16, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 16, real_data)
    points2 = get_actual_points(lineup2, 16, real_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week16_results.append({
        'Woche': 16,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_16 = [r['Sieger'] for r in week16_results]

# Woche 17: Finale
week17_matchups = [(winners_16[0], winners_16[1])]

week17_results = []
for team1, team2 in week17_matchups:
    lineup1 = get_best_lineup(team1, 17, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 17, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 17, real_data)
    points2 = get_actual_points(lineup2, 17, real_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week17_results.append({
        'Woche': 17,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Ergebnisse zusammenführen
df_playoff_results = pd.DataFrame(week15_results + week16_results + week17_results)

# Optional an bestehende Ergebnisse anhängen:
df_results = pd.concat([df_results, df_playoff_results], ignore_index=True)

# Finale anzeigen
champion = df_playoff_results[df_playoff_results['Woche'] == 17]['Sieger'].values[0]
print(f"DM-Team ist: {DM_team}")
print(f"Der Champion ist: {champion}")